## SALib Sensitivity analysis

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyNetLogo

# Import variance-based sensitivity analysis
from SALib.sample import saltelli
from SALib.analyze import sobol

In [2]:
problem = {
    'num_vars':6,
    'names': ['random-seed',
            'grass-regrowth-time',
            'sheep-gain-from-food',
            'wolf-gain-from-food',
            'sheep-reproduce',
            'wolf-reproduce'], 
  'bounds': [[1, 100000],
             [20., 40.], 
             [2., 8.], 
             [16., 32.],
             [2., 8.],
             [2., 8.]]
}

In [3]:
netlogo = pyNetLogo.NetLogoLink(gui=False)
netlogo.load_model(r'Wolf Sheep Predation_v6.nlogo')

In [4]:
n = 1000
param_values = saltelli.sample(problem, n, calc_second_order=True)

In [5]:
param_values.shape

(14000, 6)

## How to start parallel engine
Before running following cells, you need to open Anaconda Prompt, and run the following code:

ipcluster nbextension enable

ipcluster start -n 4 

In [6]:
import ipyparallel

client = ipyparallel.Client()
client.ids

[0, 1, 2, 3]

In [7]:
results = pd.DataFrame(columns=['Avg. sheep', 'Avg. wolves'])

In [8]:
direct_view = client[:]

In [9]:
def simulation(experiment):
    #Set the input parameters
    for i, name in enumerate(problem['names']):
        if name == 'random-seed':
            #The NetLogo random seed requires a different syntax
            netlogo.command('random-seed {}'.format(experiment[i]))
        else:
            #Otherwise, assume the input parameters are global variables
            netlogo.command('set {0} {1}'.format(name, experiment[i]))

    netlogo.command('setup')
    #Run for 100 ticks and return the number of sheep and wolf agents at each time step
    counts = netlogo.repeat_report(['count sheep','count wolves'], 100)    
    
    results = pd.Series([counts['count sheep'].values.mean(), 
                         counts['count wolves'].values.mean()], 
                         index=['Avg. sheep', 'Avg. wolves'])
    
    return results

In [10]:
import os

direct_view.push(dict(cwd=os.getcwd()))

<AsyncResult: _push>

In [11]:
%%px

import os
os.chdir(cwd)
import pyNetLogo
import pandas as pd

netlogo = pyNetLogo.NetLogoLink(gui=False)
netlogo.load_model(r'Wolf Sheep Predation_v6.nlogo')

In [12]:
import time

t0 = time.time()

lview = client.load_balanced_view()

results = pd.DataFrame(lview.map_sync(simulation, param_values))

elapsed = time.time() - t0


CompositeError: one or more exceptions from call to method: simulation
[3:apply]: NameError: name 'problem' is not defined
[2:apply]: NameError: name 'problem' is not defined
[1:apply]: NameError: name 'problem' is not defined
[0:apply]: NameError: name 'problem' is not defined
.... 13996 more exceptions ...

In [ ]:
print(elapsed, ' seconds')

In [ ]:
results.to_csv('Sobol_parallel.csv')

In [ ]:
results.head(5)